# Preview and Preliminary results - Women's March 2017 dataset

In [1]:
import os
import glob
import pandas as pd

In [2]:
path = '../data/'
filenames = glob.glob(os.path.join(path, '*.json'))
filenames.sort()

wm_data = pd.DataFrame()

In [3]:
wm_00 = pd.read_json(filenames[0], lines=True)
wm_01 = pd.read_json(filenames[1], lines=True)
wm_02 = pd.read_json(filenames[2], lines=True)
wm_03 = pd.read_json(filenames[3], lines=True)
wm_04 = pd.read_json(filenames[4], lines=True)
wm_05 = pd.read_json(filenames[5], lines=True)
wm_06 = pd.read_json(filenames[6], lines=True)
wm_07 = pd.read_json(filenames[7], lines=True)
wm_08 = pd.read_json(filenames[8], lines=True)
wm_09 = pd.read_json(filenames[9], lines=True)
wm_10 = pd.read_json(filenames[10], lines=True)
wm_11 = pd.read_json(filenames[11], lines=True)
wm_12 = pd.read_json(filenames[12], lines=True)
wm_13 = pd.read_json(filenames[13], lines=True)
wm_14 = pd.read_json(filenames[14], lines=True)
wm_15 = pd.read_json(filenames[15], lines=True)
wm_16 = pd.read_json(filenames[16], lines=True)

In [4]:
wm_list = [wm_00, wm_01, wm_02, wm_03, wm_04, wm_05, wm_06, wm_07, wm_08, wm_09, wm_10, \
           wm_11, wm_12, wm_13, wm_14, wm_15, wm_16]

for wm in wm_list:
    wm_data = wm_data.append(wm)

In [6]:
wm_data.head(10)

,created_at,favorite_count,followers_count,hashtags,id_str,location,mentions,name,retweet_count,screen_name,source,text,urls,verified
0,2017-01-22 14:11:52,0,162,,823171324692533248,"Austin, TX",[peterdaou],Craig Dadoly,188,craigdadoly,Twitter for iPhone,RT @peterdaou: Having participated in protests...,,False
1,2017-01-22 14:11:52,1,16,[womensmarch],823171325481144320,"Venice, Italy",,Arta,0,ArtaBudlla,Twitter for Android,"My heart and, above of all, my actions and int...",,False
2,2017-01-22 14:11:51,0,15,[WomensMarch],823171323652345856,"Waterloo, Iowa",[AntonioFrench],Patricia Smart,56742,psmart43,Twitter for Android,RT @AntonioFrench: The #WomensMarch in downtow...,https://t.co/mzb5xMZUKC,False
3,2017-01-22 14:11:53,0,288,[WomensMarch],823171331474780160,"Clemson, SC",[Nick_Offerman],Taylor Bradley,161922,vtbradley,Twitter for Android,RT @Nick_Offerman: I'm a nasty girl #WomensMar...,https://t.co/GjFriucGUY,False
4,2017-01-22 14:11:52,0,2023,[WomensMar],823171326726860800,Brazil,[chrisftboob],nági,19048,nagilacaroline,Twitter for Android,RT @chrisftboob: It doesn't make you less of a...,,False
5,2017-01-22 14:11:53,0,165,[Wome],823171329968979968,Here with you,[JimGaffigan],Cindy D,25035,cinnamazon,Twitter for Android,RT @JimGaffigan: I can’t believe every woman I...,,False
6,2017-01-22 14:11:53,0,852,[WMWYouth],823171329100840960,Brasil,[womensmarch],Laura_Miliano,3723,Laura_Miliano,Twitter Web Client,RT @womensmarch: Look out for our Youth Ambass...,,False
7,2017-01-22 14:11:54,0,517,[WomensMarch],823171332464672768,,,Laboris Gloria Ludi,0,wraithburn,Twitter for Android,"Unlike the #WomensMarch that I saw, she's actu...",https://t.co/9F8sZ2Ln5n,False
8,2017-01-22 14:11:52,0,2402,[WomensMarch],823171326965739520,"Las Vegas, Nevada",[donnabrazile],Susan,167,SusanHenshaw50,Twitter for iPhone,RT @donnabrazile: “Wade in the water...God’s g...,,False
9,2017-01-22 14:11:52,0,38,,823171325187526656,,"[CornellWBrooks, SenatorSessions]",jamiepike,3027,jamiepike,Twitter for iPhone,RT @CornellWBrooks: If @SenatorSessions wouldn...,,False


In [7]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
num_of_tweets = wm_data.shape[0]
users_count = wm_data['screen_name'].nunique()

print('Total number of tweets retriven:', num_of_tweets)
print('Number of unique numbers:', users_count)
print('Ratio of average tweets per user:', num_of_tweets/users_count)

Total number of tweets retriven: 9170486
Number of unique numbers: 2438903
Ratio of average tweets per user: 3.760086399500103


In [ ]:
fig, axes = plt.subplots(figsize=(18,6))
sns.countplot(x=wm_data['created_at'], order=wm_data['created_at'].value_counts().index)
plt.title('Tweets timeline')
axes.set(xlabel='Date time', ylabel='# of tweets') 
plt.show()

In [ ]:
top_fave = wm_data.sort_values('favorite_count', ascending=False).head().reset_index(drop=True).head()
top_rts = wm_data.sort_values('retweets_count', ascending=False).head().reset_index(drop=True).head()